In [8]:
import itertools
import numpy as np
from gurobipy import Model, GRB

def solve_linear_program():
    model = Model("Linear_Programming_Example")
    
    attributes = ["A", "B", "C", "D", "E", "F", "G", "H"]
    relations = [["A","B","C","D"], ["A","B","E","F"], ["C","D","E","F"], ["A","B","G","H"], ["C","D","G","H"]]
    variables = {}
    
    for i in range(1, len(attributes) + 1):
        for subset in itertools.combinations(attributes, i):
            rho = "".join(subset)
            sigma = "".join(subset)
            name = f"D({rho}||{sigma})"
            # ub=len(subset)
            x = model.addVar(lb=0, ub=np.log2(len(subset)), name=name)
            variables[frozenset(subset)] = x
    
    objective = variables[frozenset(attributes)]   
    for relation in relations:
        objective -= variables[frozenset(relation)]
    
    model.setObjective(objective, GRB.MINIMIZE)
        
    #fds = [[["E","F","G","H"], ["A","B","C","D"]], [["A","B","C","E"], ["D","F","G","H"]], [["B","C","D","G"], ["A","E","F","H"]]]
    #candidate_keys = [[["E","F","G","H"], ["A","B","C","D", "E","F","G","H"]], [["A","B","C","E"], ["D","F","G","H", "A","B","C","E"]], [["B","C","D","G"], ["A","E","F","H", "B","C","D","G"]]]
    #fds = fds + candidate_keys
    #for Y, x in fds:
    #    Yx = Y + x
    #    model.addConstr(variables[frozenset(Yx)] == variables[frozenset(Y)], "fd_" + "".join(Y) + "->" + "".join(x))
    
    # Monotonicty of relative entropy
    for subset in variables:
        for i in range(1, len(subset)):
            for subset_prime in itertools.combinations(subset, i):
                model.addConstr(variables[frozenset(subset_prime)] <= variables[subset], "monotonicity_" + "".join(subset_prime) + "->" + "".join(subset))
    # Save model to lp file
    model.write("relative_entropy_program1.lp")

    # Optimize the model
    model.optimize()

    # Check optimization status
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective value = {model.objVal}")
        return variables[frozenset(attributes)].x
    elif model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("The model is unbounded.")
    else:
        print(f"Optimization ended with status {model.status}")
    
solution = solve_linear_program()
print(solution)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6050 rows, 255 columns and 12100 nonzeros
Model fingerprint: 0x392bdbeb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 5915 rows and 217 columns
Presolve time: 0.01s
Presolved: 135 rows, 38 columns, 270 nonzeros


CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6050 rows, 255 columns and 12100 nonzeros
Model fingerprint: 0x392bdbeb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+00]
  RHS r